In [1]:
# Need nltk installed
# No need to run this part

import nltk
from nltk.corpus import stopwords

In [2]:
import numpy as np
import os
import os.path
import string
import re
import operator
import csv
import logging
import time
import pickle

In [3]:
# Need gensim package installed
# No need to run this

import gensim
import gensim.models as g

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [4]:
# Need PyLucene installed
# No need to run this

import lucene

from java.io import File 
from org.apache.lucene.analysis.standard import StandardAnalyzer 
from org.apache.lucene.document import Document, Field 
from org.apache.lucene.index import IndexWriter, IndexWriterConfig 
from org.apache.lucene.store import SimpleFSDirectory 
from org.apache.lucene.util import Version
from org.apache.lucene.search import IndexSearcher 
from org.apache.lucene.index import IndexReader 
from org.apache.lucene.queryparser.classic import QueryParser 

from org.apache.lucene import document, store, util


In [5]:
# Need Keras installed
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, Embedding, LSTM, merge
from keras.regularizers import l2, activity_l2

Using Theano backend.


# Doc2Vec

## To get all documents in one file for input to Doc2Vec
file already generated(allText.txt) - no need to run this

In [3]:
#dont forget to remove the allText.txt file everytime you run this code.
dirName = "/home/tarun/PE/corpus/"
uniqueWords = {}
allText = ""
l = os.listdir(dirName)
for i in range(len(l)):
    l[i] = int(l[i][:-4])

l = sorted(l)
for i in range(len(l)):
    l[i] = str(l[i]) + ".txt"

for fname in l:
    with open(os.path.join(dirName, fname), 'r') as myFile:
        fileText = myFile.read().replace('\n', ' ') #each file's text - a paragraph like structure for us
        for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
            fileText = fileText.replace(char, '')
        fileText = fileText.replace("displaystyle", '')
        fileText = re.sub("""displaystyle""", "", fileText, re.I|re.S)
        fileText = re.sub("""[^0-9A-Za-z]""", ' ', fileText)

        fileText = re.sub("""\s+""", " ", fileText)
        fileText = re.sub("""\t+""", " ", fileText)
        allText = allText + fileText
        allText = allText + "\n"
    with open(os.path.join(dirName, fname), 'w') as myFile:
        myFile.write(fileText)
with open(os.path.join("/home/tarun/PE/", "allTextNormal.txt"), 'w') as myFile:
    myFile.write(allText)

## Check the number of cores (to be given as paramter in the Doc2Vec training)

In [7]:
import multiprocessing

cores = multiprocessing.cpu_count()
assert g.doc2vec.FAST_VERSION > -1,
print cores

SyntaxError: invalid syntax (<ipython-input-7-d7bf84e289eb>, line 4)

In [ ]:
fName = "/home/tarun/PE/allText.txt"
uniqueWords = {}
with open(fName, 'r') as myFile:
    for line in myFile:
        #print line
        #print "\n\n\n"
        sentence = line.split()
        #print sentence
        #print "\n\n\n"
        
        for eachWord in sentence:
            eachWord = eachWord.lower()
            if(eachWord.isdigit() == False):
                try:
                    if(uniqueWords[eachWord]):
                        uniqueWords[eachWord] = uniqueWords[eachWord] + 1
                except:
                    uniqueWords[eachWord] = 1
print len(uniqueWords)

In [ ]:
sorted_x = sorted(uniqueWords.items(), key=operator.itemgetter(1), reverse = True)

In [ ]:
topUnique = dict(sorted(uniqueWords.iteritems(), key=operator.itemgetter(1), reverse=True)[:5])
print topUnique

In [ ]:
fName = "/home/tarun/PE/allText.txt"
uniqueWords = {}
with open(fName, 'r') as myFile:
    for line in myFile:
        #print line
        #print "\n\n\n"
        sentence = line.split()
        #print sentence
        #print "\n\n\n"
        for eachWord in sentence:
            eachWord = eachWord.lower()
            if(eachWord.isdigit() == False and eachWord not in stopwords.words('english')):
                try:
                    if(uniqueWords[eachWord]):
                        uniqueWords[eachWord] = uniqueWords[eachWord] + 1
                except:
                    uniqueWords[eachWord] = 1
print len(uniqueWords)

In [ ]:
topUnique = dict(sorted(uniqueWords.iteritems(), key=operator.itemgetter(1), reverse=True)[:5])
print topUnique

# Doc To Vec

In [7]:
tic = time.time()
#doc2vec parameters
vector_size = 300
window_size = 8
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 8 #number of parallel processes - number of cores

#pretrained word embeddings
#pretrained_emb = None #None if use without pretrained embeddings

#input corpus
train_corpus = "/home/tarun/PE/allText.txt"

#output model
saved_path = "/home/tarun/PE/doc2vec/model3.bin"

#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#train doc2vec model
docs = g.doc2vec.TaggedLineDocument(train_corpus)
model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1) #model3
#model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dm_concat=1) #model2

#save model
model.save(saved_path)
toc = time.time()
print toc - tic
#model2 - without dbow_words
#model3 - with

391.717040062


In [54]:
# PATHS 
luceneIndexPath = '/home/tarun/PE/lucene/luceneIndexDirectory/'
corpus = '/home/tarun/PE/corpus/'
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

#parameters
modelPath="/home/tarun/PE/doc2vec/model3.bin"
#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

#load model
#model = g.Doc2Vec.load(modelPath)
#test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r").readlines() ]
#for d in test_docs:
    #output.write( " ".join([str(x) for x in m.infer_vector(d, alpha=start_alpha, steps=infer_epoch)]) + "\n" )

#output.flush()
#output.close()
    
lucene.initVM()

# ANALYZER
analyzer = StandardAnalyzer(util.Version.LUCENE_CURRENT) 

# DIRECTORY
directory = SimpleFSDirectory(File(luceneIndexPath))


#dont forget to remove the luceneIndexDirectory file everytime you run this code.

'''
# INDEX WRITER
writerConfig = IndexWriterConfig(util.Version.LUCENE_CURRENT, analyzer) 
writer = IndexWriter(directory, writerConfig)

l = os.listdir(corpus)
for i in range(len(l)):
    l[i] = int(l[i][:-4])

l = sorted(l)
for i in range(len(l)):
    l[i] = str(l[i]) + ".txt"

#print writer.numDocs()
# INDEXING ALL DOCUMENTS/ARTICLES IN THE CORPUS
for fileName in l:
    #print fileName
    document = Document()
    article = os.path.join(corpus, fileName)
    content = open(article, 'r').read()
    document.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
    writer.addDocument(document)
#print writer.numDocs()
writer.close()
'''

# INDEX READER
reader = IndexReader.open(directory)
searcher = IndexSearcher(reader)

# QUERYING FOR A QUESTION
queryParser = QueryParser(util.Version.LUCENE_CURRENT, "text", analyzer)

question = "When athletes begin to exercise, their heart rates and respiration rates increase.  At what level of organization does the human body coordinate these functions?"
query = queryParser.parse(queryParser.escape(question))
hits = searcher.search(query, 3)
'''
for h in hits:
    hit = lucene.Hit.cast_(h)
    #id, doc = hit.getId(), hit.getDocument()
    doc_id = hit.getId()
    print doc_id
    model.docvecs[doc_id] 
'''
for hit in hits.scoreDocs:
    print hit.score, hit.doc, hit.toString()
    #doc = searcher.doc(hit.doc)
    #maddy = doc.get("text").encode("utf-8")
    #print maddy


0.388097137213 1568 doc=1568 score=0.38809714 shardIndex=-1
0.266801327467 1600 doc=1600 score=0.26680133 shardIndex=-1
0.249090507627 1745 doc=1745 score=0.24909051 shardIndex=-1


In [59]:
modelPath="/home/tarun/PE/doc2vec/model2.bin"
model = g.Doc2Vec.load(modelPath)

In [101]:

#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter="\t")
    header = 0
    storeInputVecInFile = np.zeros([(numQuestions*4),900])
    storeOutputVecInFile = np.zeros([(numQuestions*4),1])
    inputNum = 0
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            question = row[1]
            query = queryParser.parse(queryParser.escape(question))
            questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
            numPages = 3
            hits = searcher.search(query, numPages)
            docVec = np.zeros(300)
            output = 0
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                docV = model.docvecs[doc_id]
                docVec = docVec + docV
            docVec = docVec/numPages 
            for option in [row[3], row[4], row[5], row[6]]:
                optionVec = model.infer_vector(option, alpha=start_alpha, steps=infer_epoch)
                if(option == row[2]):
                    output = 1
                else:
                    output = 0
                #inputVec = merge([docVec, questionVec, optionVec], mode='concat')
                inputVec = np.concatenate([docVec, questionVec, optionVec])
                storeInputVecInFile[inputNum] = inputVec
                storeOutputVecInFile[inputNum] = output
                inputNum = inputNum + 1


np.save('input.npy', storeInputVecInFile)
np.save('output.npy', storeOutputVecInFile)
toc = time.time()
print toc - tic

3413.68172193


In [4]:
storeInputVecInFile = np.load('input.npy')
storeOutputVecInFile = np.load('output.npy')

In [96]:
# TESTING
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

numQuestions = 0
for row in test:
    numQuestions = numQuestions + 1

storeInputVecInFileTest = np.zeros([(numQuestions*4),900])
    
answers = ['A', 'B', 'C', 'D']
checkRows = []
inputNum = 0
for row in test:
    question = row[1]
    query = queryParser.parse(queryParser.escape(question))
    questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
    numPages = 3
    hits = searcher.search(query, numPages)
    docVec = np.zeros(300)
    for hit in hits.scoreDocs:
        doc_id = hit.doc
        docV = model.docvecs[doc_id]
        docVec = docVec + docV
    docVec = docVec/numPages 
    answerScores = []
    for option in [row[2], row[3], row[4], row[5]]:
        optionVec = model.infer_vector(option, alpha=start_alpha, steps=infer_epoch)
        inputVec = np.concatenate([docVec, questionVec, optionVec])
        storeInputVecInFileTest[inputNum] = inputVec
        inputNum = inputNum + 1


KeyboardInterrupt: 

In [5]:
storeInputVecInFile = np.load('input.npy')
storeOutputVecInFile = np.load('output.npy')

In [54]:
network = Sequential()
#network.add(Dense(output_dim=300, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(output_dim=300, input_dim=900, init='uniform', activation='sigmoid'))
network.add(Dense(200, init='uniform', activation='sigmoid'))
network.add(Dense(50, init='uniform', activation='sigmoid'))
#network.add(Dense(50, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(1, init='uniform', activation='sigmoid', bias=True))
network.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
#both these codes should be run together, otherwise the network has already been learnt.
#tic = time.time()
network.fit(storeInputVecInFile, storeOutputVecInFile, nb_epoch = 1, batch_size = 50, verbose = 1)
#verbose = 2 is needed otherwise it gives error of I/O operations on a closed file
#toc = time.time()
#print toc - tic

Epoch 1/1
10000/10000 [==============================] - 1s - loss: 0.1255 - acc: 1.0000     


In [33]:
tic = time.time()
storeInputVecInFileTest = np.load('inputTest.npy')
ans = network.predict(storeInputVecInFileTest, batch_size=2500, verbose=0)
#changing batch_size does not make a difference. Tried with 2500, 1, 50 
#ans1 = network.predict_proba(storeInputVecInFileTest, batch_size=1, verbose=0)
toc = time.time()
print (toc - tic)

8.76206493378


In [34]:
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

numQuestions = 0
for row in test:
    numQuestions = numQuestions + 1

    
submissionFile = open("doc2vecModelTest.csv", "w")
writer = csv.writer(submissionFile, delimiter=',')
writer.writerow(['id', 'correctAnswer'])
qIds = []
answers = ['A', 'B', 'C', 'D']
for row in test:
    questionId = row[0]
    qIds.append(questionId)
    
for q in range(numQuestions):
    answerScores = []
    opAProb = ans[q*4]
    opBProb = ans[q*4 + 1]
    opCProb = ans[q*4 + 2]
    opDProb = ans[q*4 + 3]
    answerScores.append(opAProb)
    answerScores.append(opBProb)
    answerScores.append(opCProb)
    answerScores.append(opDProb)
    writer.writerow([qIds[q] , answers[answerScores.index(np.max(answerScores))]])
    
submissionFile.close()

In [37]:
modelPath="/home/tarun/PE/doc2vec/model2.bin"
modelLoad = g.Doc2Vec.load(modelPath)

In [42]:
#print modelLoad.most_similar('2')
#print "\n"
print modelLoad.similarity("heart", "blood")
print "\n"
print modelLoad.most_similar("blood")
print "\n"
print modelLoad.docvecs.most_similar(2)
#del modelLoad

0.85245965204


[(u'lungs', 0.9407699108123779), (u'bloodstream', 0.9289163947105408), (u'pumped', 0.9040083885192871), (u'air', 0.8987337946891785), (u'oxygenated', 0.8982198238372803), (u'liver', 0.8960069417953491), (u'gastroderm', 0.8852612972259521), (u'hemoglobin', 0.8786916732788086), (u'anoxic', 0.8705736398696899), (u'inside', 0.8683980107307434)]


[(371, 0.9690296649932861), (1247, 0.9506374597549438), (366, 0.932389497756958), (128, 0.9301618337631226), (552, 0.9229782819747925), (1487, 0.9073729515075684), (46, 0.8987748622894287), (507, 0.8912057876586914), (1439, 0.8866207003593445), (912, 0.8786807060241699)]


In [49]:
d = model.docvecs[2324]
k = model.docvecs[2321] 

In [53]:
docVecs = np.empty([2,2])
print docVecs
docVec = np.empty([2,2])
print docVec
docVecs = docVecs + docVec
print docVecs

[[  6.92552414e-310   6.92552414e-310]
 [  1.23932454e-316   1.23931980e-316]]
[[ 0.  0.]
 [ 0.  0.]]
[[  6.92552414e-310   6.92552414e-310]
 [  1.23932454e-316   1.23931980e-316]]
